
# 1 - Imports and defining functions

In [ ]:
import numpy as np

from pyFM.mesh import TriMesh
from pyFM.functional import FunctionalMapping

import meshplot as mp

def plot_mesh(myMesh,cmap=None):
    mp.plot(myMesh.vertlist, myMesh.facelist,c=cmap)
    
def double_plot(myMesh1,myMesh2,cmap1=None,cmap2=None):
    d = mp.subplot(myMesh1.vertlist, myMesh1.facelist, c=cmap1, s=[2, 2, 0])
    mp.subplot(myMesh2.vertlist, myMesh2.facelist, c=cmap2, s=[2, 2, 1], data=d)

def visu(vertices):
    min_coord,max_coord = np.min(vertices,axis=0,keepdims=True),np.max(vertices,axis=0,keepdims=True)
    cmap = (vertices-min_coord)/(max_coord-min_coord)
    return cmap

# 2- Loading and processing a mesh

### Basic Mesh methods

A TriMesh class can be created from a path (to a .off or a .obj file) or simply an array of vertices and an optional array of faces.

The mesh can be centered, area-normalized, rotated or translated when loading.


Vertices and faces are stored in the 'vertlist' and 'facelist' attributes. One can also use 'mesh.vertices' and 'mesh.faces' to access them. While these notations can feel non-intuitive they result in clearer functions as it avoids expressions of the form ```mesh.vertices - vertices```.

A TriMesh class possess multiple attributes like edges, per-face area, per-vertex area, per-face normals, per-vertex normals, ...

In [ ]:
mesh1 = TriMesh('data/2/target.off', area_normalize=True, center=False)
mesh2 = TriMesh(mesh1.vertlist, mesh1.facelist)

In [ ]:
# Attributes are computed on the fly and cached
edges = mesh1.edges

area = mesh1.area

face_areas = mesh1.face_areas
vertex_areas = mesh1.vertex_areas
face_normals = mesh1.normals

# AREA WEIGHTED VERTEX NORMALS
vertex_normals_a = mesh1.vertex_normals

# UNIFORM WEIGHTED VERTEX NORMALS
mesh1.set_vertex_normal_weighting('uniform')
vertex_normals_u = mesh1.vertex_normals

### Geodesics

We propose three versions to compute geodesics :
- Heat method - based on [potpourri3d](https://github.com/nmwsharp/potpourri3d) using robust laplacian (recommended)
- Heat method - pure python implementation from pyFM (not robust but control on the whole code)
- Dijkstra

In [ ]:
# Geodesic distance from a given index
# Set robust to False to obtain result from the Python implementation
dists = mesh1.geod_from(1000, robust=True)

In [ ]:
#S1_geod = mesh1.get_geodesic(verbose=True)

### Laplacian and functions

The spectrum of the LBO can be computed easily.

Eigenvalues and eigenvectors are stored in the ```mesh.eigenvalues``` and ```mesh.eigenvectors``` attributes.

Gradient and divergence can be computed using the associated methods. Using the ```mesh.project``` and ```mesh.unproject``` functions allows to switch between seeing a function in the LBO basis or on the complete shape.

The squared $L^2$ norm and $H^1_0$ norm can be computed via the ```mesh.l2_sqnorm``` and ```mesh.h1_sqnorm``` methods.

In [ ]:
# By default does not use the intrinsic delaunay Laplacian
mesh1.process(k=100, intrinsic=False, verbose=True);

In [ ]:
# set up a figure twice as wide as it is tall
i = 0
d = mp.subplot(mesh1.vertlist, mesh1.facelist, c=mesh1.eigenvectors[:,i], s=[1, 10, 0])
for i in range(1,10):
    d = mp.subplot(mesh1.vertlist, mesh1.facelist, c=mesh1.eigenvectors[:,i], s=[1, 10, i], data=d)

#d.save("myplottest")

# 3 - Computing the functional map

**Loading data (1/2)**

In [ ]:
mesh1 = TriMesh('data/2/target.off')

**Loading data (2/2)**

In [ ]:
#♦mesh2 = TriMesh('data/2022-11-23-CT/1_cropped_scaled_freeze_translated.off')
mesh2 = TriMesh('data/2/source.off')
#mesh2 = TriMesh('data/2022-11-23-CT/CT_cleaned_03_removed_isolated_decimated.off')

**Displaying data**

In [ ]:
print(f'Mesh 1 : {mesh1.n_vertices:4d} vertices, {mesh1.n_faces:5d} faces\n'
      f'Mesh 2 : {mesh2.n_vertices:4d} vertices, {mesh2.n_faces:5d} faces')

d = mp.subplot(mesh1.vertlist, mesh1.facelist, None, s=[2, 2, 0])
mp.subplot(mesh2.vertlist, mesh2.facelist, None, s=[2, 2, 1], data=d)



**Add correspondences**

In [ ]:
loadLandmark = True
if loadLandmark:
    landmarks = np.loadtxt('data/18/landmarks.txt',dtype=int)[:8]  # loading 5 landmarks
    display(landmarks)

    landmarks_plot_CT = TriMesh(mesh1.vertlist[landmarks[:,0]])
    landmarks_plot_RGBD = TriMesh(mesh2.vertlist[landmarks[:,1]])

    #landmarks_plot_CT = TriMesh(mesh1.facelist[landmarks[:,0]])
    #landmarks_plot_RGBD = TriMesh(mesh2.facelist[landmarks[:,1]])

    print('Retrieved matching faces from CT & RGBD pointclouds')
    
else:
     landmarks = []



#d = mp.subplot(mesh1.vertlist, mesh1.facelist, None, s=[2, 2, 0])

p1 = mp.plot(mesh1.vertlist, mesh1.facelist, None)
p2 = mp.plot(mesh2.vertlist, mesh2.facelist, None)

if loadLandmark:
    color = c=np.random.rand(*landmarks_plot_CT.vertlist.shape)
    p1.add_points(landmarks_plot_CT.vertlist, c = color, shading={"point_size": 0.3})
    p2.add_points(landmarks_plot_RGBD.vertlist, c = color, shading={"point_size": 0.3})

#i = 0
#for vert in landmarks_plot_CT.vertlist:
#    color = c=np.random.rand(3)
#    p1.add_points(landmarks_plot_CT.vertlist[i], shading={"point_size": 15, "point_color": color})
#    p2.add_points(landmarks_plot_RGBD.vertlist[i], shading={"point_size": 15, "point_color": color})
#    i = i+1

#p = mp.plot(landmarks_plot_CT.vertlist, landmarks_plot_CT.facelist, c = None, shading={"point_size": 15})
#mp.subplot(landmarks_plot_RGBD.vertlist, landmarks_plot_RGBD.facelist, None, s=[2, 2, 3], data=d, shading={"point_size": 15})

#d = mp.subplot(mesh1.vertlist, mesh1.facelist, c=mesh1.eigenvectors[:,i], s=[1, 10, i], data=d)



**Computing descriptors**

In [ ]:
process_params = {
    'n_ev': (25,25),  # Number of eigenvalues on source and Target
    'landmarks': landmarks,  # loading 5 landmarks
    'subsample_step': 5,  # In order not to use too many descriptors
    'descr_type': 'HKS',  # WKS or HKS
}


model = FunctionalMapping(mesh1,mesh2)
model.preprocess(**process_params,verbose=True);

**Fitting the model**

$\newcommand{\RR}{\mathbb{R}}$
$\newcommand{\Ss}{\mathcal{S}}$
$\newcommand{\uargmin}[1]{\underset{#1}{\text{argmin}}\;}$
$\newcommand{\uargmax}[1]{\underset{#1}{\text{argmax}}\;}$
$\def\*#1{\mathbf{#1}}$

In pyFM, we always consider functional maps $\*C:\Ss_1\to\Ss_2$ and pointwise maps $T:\Ss_2\to\Ss_1$ going in opposite directions, with $\*C$ always going from shape 1 to shape 2 !

Optimization problem is
\begin{equation}
\uargmin{\*C\in\RR^{k_2\times k_1}} w_{descr}\|\*C\*A - \*B\|^2 + w_{lap}\|\*C\Delta_1 - \Delta_2\*C\|^2 + w_{\text{d- comm}}\sum_i \|\*C\Gamma_1^i - \Gamma_2^i\*C\|^2 + w_{\text{orient}}\sum_i \|\*C\Lambda_1^i - \Lambda_2^i\*C\|^2
\end{equation}

with $\Gamma_1^i$ and $\Gamma_2^i$ [multipliative operators](http://www.lix.polytechnique.fr/~maks/papers/fundescEG17.pdf) associated to the $i$-th descriptors, $\Lambda_1^i$ and $\Lambda_2^i$ [orientation preserving operators](https://arxiv.org/abs/1806.04455) associated to the $i$-th descriptors

In [ ]:
fit_params = {
    'w_descr': 1e0, #scaling for the descriptor preservation term
    'w_lap': 1e-2, #scaling of the laplacian commutativity term
    'w_dcomm': 1e-1, #scaling of the multiplicative operator commutativity
    'w_orient': 0, #scaling of the orientation preservation term
    #'orient_reversing':True, #Whether to use the orientation reversing term instead of the orientation preservation one
    #'optinit':'zeros' #Initialization

}



model.fit(**fit_params, verbose=True)

**Visualizing the associated point to point map**

In [ ]:
p2p_21 = model.get_p2p(n_jobs=1)
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21]
double_plot(mesh1,mesh2,cmap1,cmap2)

# 4 - Refining the Functional Map
```model.FM``` returns the current state of functional map. One can change which one is returned by using ```model.change_FM_type(FM_type)```, as one can see below. 

**ICP**

In [ ]:
model.icp_refine(verbose=True)
p2p_21_icp = model.get_p2p()
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21_icp]
double_plot(mesh1,mesh2,cmap1,cmap2)

**Zoomout**

In [ ]:
model.change_FM_type('classic') # We refine the first computed map ('classic') or the icp-refined one ('icp')
model.zoomout_refine(nit=15, step = 1, verbose=True)
print(model.FM.shape)
p2p_21_zo = model.get_p2p()
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21_zo]
double_plot(mesh1,mesh2,cmap1,cmap2)

In [ ]:
# open file in write mode
with open(r'p2p_icp.txt', 'w') as p2p_icp:
    for item in p2p_21_icp:
        # write each item on a new line
        p2p_icp.write("%s\n" % item)
    print('Done')

# open file in write mode
with open(r'p2p_zo.txt', 'w') as f_export_zo:
    for item in p2p_21_zo:
        # write each item on a new line
        f_export_zo.write("%s\n" % item)
    print('Done')
